In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Step_back"

# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

In [3]:
import langchain
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.runnables import RunnableLambda
from langsmith import traceable
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [4]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## Reloading basic NVIDIA Vectorstore

In [5]:
# Load the index back

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
new_vector_store = FAISS.load_local(
    "nvidia_large_baseline", 
    embeddings,  # Use the same embeddings used during creation
    allow_dangerous_deserialization=True  # Important for loading
)


retriever = new_vector_store.as_retriever()

In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=4000,
)

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large"
)

In [8]:
# Few Shot Examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "what can the members of The Police do?",
    },
    {
        "input": "Jan Sindel’s was born in what country?",
        "output": "what is Jan Sindel’s personal history?",
    },
]
# We now transform these to example messages
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [9]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert at world knowledge. Your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer. Here are a few examples:""",
        ),
        # Few shot examples
        few_shot_prompt,
        # New question
        ("user", "{question}"),
    ]
)

In [10]:
question_gen = prompt | llm | StrOutputParser()

In [11]:
question = "was chatgpt around while trump was president?"

### Example step-back question generation

In [12]:
question_gen.invoke({"question": question})

'when was ChatGPT developed and released?'

## RAG pipeline 

In [13]:
def setup_rag_pipeline():
    """
    Creates a chain that:
      - takes 'question' as input
      - injects it + 'context' into a prompt
      - runs it through the LLM
      - returns the text of the LLM response
    """
    template = """
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    prompt_for_rag = ChatPromptTemplate.from_template(template)
    
    return (
        {
            "context": RunnableLambda(lambda x: x["retrieved_contexts"]), 
            "input": RunnableLambda(lambda x: x["question"])  
        }
        | prompt_for_rag
        | llm
        # The model returns an AIMessage; we retrieve its content (string) here
        | RunnableLambda(lambda msg: msg.content)
    )

rag_chain = setup_rag_pipeline()

In [14]:
def query_rag(question: str, top_k=4):
    """
    1. Retrieves top-k chunks for the original question (normal).
    2. Retrieves top-k chunks for the "step-back" paraphrased question.
    3. Displays all retrieved docs in a DataFrame.
    4. Runs the RAG pipeline using all retrieved texts.
    """
    # Step 5a. Retrieve top-k chunks for normal question
    normal_docs_with_scores = retriever.vectorstore.similarity_search_with_score(question, k=top_k)
    
    # Step 5b. Retrieve top-k chunks for step-back question
    step_back_question = question_gen.invoke({"question": question})
    print(step_back_question)
    step_back_docs_with_scores = retriever.vectorstore.similarity_search_with_score(step_back_question, k=top_k)
    
    # Convert normal retrieval results to a DataFrame
    df_normal = pd.DataFrame([
        {
            "Query Type": "Normal",
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),
            "Retrieved Chunk": doc.page_content[:1000] + "..."
        }
        for doc, score in normal_docs_with_scores
    ])
    
    # Convert step-back retrieval results to a DataFrame
    df_step_back = pd.DataFrame([
        {
            "Query Type": "Step-Back",
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
            "Similarity Score": round(score, 4),
            "Retrieved Chunk": doc.page_content[:1000] + "..."
        }
        for doc, score in step_back_docs_with_scores
    ])
    
    # Combine both DataFrames
    df = pd.concat([df_normal, df_step_back], ignore_index=True)
    
    display(df)
    
    # Display the DataFrame (using a custom "ace_tools" display function)
#     import ace_tools as tools
#     tools.display_dataframe_to_user(name="Retrieved Documents", dataframe=df)
    
    # Step 5c. Combine all retrieved docs for final answer generation
#     retrieved_contexts = [doc.page_content for doc, _ in (normal_docs_with_scores + step_back_docs_with_scores)]
    
    
    
    # with this we are removing duplicates 
    
    seen = set()
    deduped_contexts = []
    for doc, _ in (normal_docs_with_scores + step_back_docs_with_scores):
        if doc.page_content not in seen:
            deduped_contexts.append(doc.page_content)
            seen.add(doc.page_content)
    
    # Step 5d. Run the RAG pipeline with the question + combined contexts
    answer = rag_chain.invoke({"question": question, "retrieved_contexts": deduped_contexts})
    
    return {
        "answer": answer,          # Now it's just a string, from `msg.content`
        "contexts": deduped_contexts
    }

In [15]:
result = query_rag("When did the CEO join the company?")

what is the timeline of the CEO's tenure at the company?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,11,1.1731,"Debora Shoquist 69 Executive Vice President, O..."
1,Normal,NVIDIA_Report_2024.pdf,48,1.2292,"/s/ PricewaterhouseCoopers LLP\nSan Jose, Cali..."
2,Normal,NVIDIA_Report_2024.pdf,11,1.2431,"supports diverse hiring, retention, and employ..."
3,Normal,NVIDIA_Report_2024.pdf,12,1.2514,"Puri previously held marketing, management con..."
4,Step-Back,NVIDIA_Report_2024.pdf,11,1.1523,"supports diverse hiring, retention, and employ..."
5,Step-Back,NVIDIA_Report_2024.pdf,11,1.1776,"Debora Shoquist 69 Executive Vice President, O..."
6,Step-Back,NVIDIA_Report_2024.pdf,12,1.2872,from Kansas State University and a B.S. degree...
7,Step-Back,NVIDIA_Report_2024.pdf,89,1.2886,"“Periodic Report”), fully complies with the re..."


In [16]:
result["answer"]

"Jen-Hsun Huang co-founded NVIDIA in 1993 and has served as the President and Chief Executive Officer since the company's inception."

In [17]:
len(result["contexts"])

6

## RAG Evaluation Dataset

In [18]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [19]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_7370/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [20]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag(example["question"])
    return {"answer": response["answer"]}

In [21]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [22]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [23]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [45]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="Step_back",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Step_back-d3deef00' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=26c2ff48-b3da-41be-84b6-a3414fe0f88f




0it [00:00, ?it/s]

what industries does NVIDIA operate in?
who leads NVIDIA as its chief executive?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,4,0.5249,infotainment systems; and Omniverse Enterprise...
1,Normal,NVIDIA_Report_2024.pdf,3,0.6685,serve. While the computing requirements of the...
2,Normal,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers..."
3,Normal,NVIDIA_Report_2024.pdf,6,0.7180,layers. This full-stack innovation approach al...
4,Step-Back,NVIDIA_Report_2024.pdf,4,0.6737,infotainment systems; and Omniverse Enterprise...
5,Step-Back,NVIDIA_Report_2024.pdf,4,0.6748,"Professional artists, architects and designers..."
6,Step-Back,NVIDIA_Report_2024.pdf,33,0.7028,NVIDIA pioneered accelerated computing to help...
7,Step-Back,NVIDIA_Report_2024.pdf,77,0.7338,The Graphics segment includes GeForce GPUs for...


how can we compare the revenues of NVIDIA over different years?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, O..."
1,Normal,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employ..."
2,Normal,NVIDIA_Report_2024.pdf,12,0.8722,"Table of Contents\nstrategy, planning, reporti..."
3,Normal,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management con..."
4,Step-Back,NVIDIA_Report_2024.pdf,11,0.5533,"Debora Shoquist 69 Executive Vice President, O..."
5,Step-Back,NVIDIA_Report_2024.pdf,11,0.6930,"supports diverse hiring, retention, and employ..."
6,Step-Back,NVIDIA_Report_2024.pdf,12,0.8750,"Table of Contents\nstrategy, planning, reporti..."
7,Step-Back,NVIDIA_Report_2024.pdf,12,0.9219,"Puri previously held marketing, management con..."


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,35,0.5506,Revenue for fiscal year 2024 was $60.9 billion...
1,Normal,NVIDIA_Report_2024.pdf,35,0.6613,other NVIDIA RTX technologies.\nProfessional V...
2,Normal,NVIDIA_Report_2024.pdf,49,0.6642,Table of Contents\nNVIDIA Corporation and Subs...
3,Normal,NVIDIA_Report_2024.pdf,35,0.7459,Table of Contents\nFiscal Year 2024 Summary\n ...
4,Step-Back,NVIDIA_Report_2024.pdf,35,0.7436,Revenue for fiscal year 2024 was $60.9 billion...
5,Step-Back,NVIDIA_Report_2024.pdf,49,0.7951,Table of Contents\nNVIDIA Corporation and Subs...
6,Step-Back,NVIDIA_Report_2024.pdf,38,0.8416,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,..."
7,Step-Back,NVIDIA_Report_2024.pdf,35,0.8704,Table of Contents\nFiscal Year 2024 Summary\n ...


### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [24]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [25]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

In [48]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="Step_back",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Step_back-daae6d53' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=8a041218-9f19-4b50-874e-c00cafa60059




0it [00:00, ?it/s]

what industries does NVIDIA operate in?
how can we compare the revenues of NVIDIA over different years?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,4,0.5249,infotainment systems; and Omniverse Enterprise...
1,Normal,NVIDIA_Report_2024.pdf,3,0.6685,serve. While the computing requirements of the...
2,Normal,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers..."
3,Normal,NVIDIA_Report_2024.pdf,6,0.7180,layers. This full-stack innovation approach al...
4,Step-Back,NVIDIA_Report_2024.pdf,4,0.6737,infotainment systems; and Omniverse Enterprise...
5,Step-Back,NVIDIA_Report_2024.pdf,4,0.6748,"Professional artists, architects and designers..."
6,Step-Back,NVIDIA_Report_2024.pdf,33,0.7028,NVIDIA pioneered accelerated computing to help...
7,Step-Back,NVIDIA_Report_2024.pdf,77,0.7338,The Graphics segment includes GeForce GPUs for...


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,35,0.5506,Revenue for fiscal year 2024 was $60.9 billion...
1,Normal,NVIDIA_Report_2024.pdf,35,0.6613,other NVIDIA RTX technologies.\nProfessional V...
2,Normal,NVIDIA_Report_2024.pdf,49,0.6642,Table of Contents\nNVIDIA Corporation and Subs...
3,Normal,NVIDIA_Report_2024.pdf,35,0.7459,Table of Contents\nFiscal Year 2024 Summary\n ...
4,Step-Back,NVIDIA_Report_2024.pdf,35,0.7436,Revenue for fiscal year 2024 was $60.9 billion...
5,Step-Back,NVIDIA_Report_2024.pdf,49,0.7951,Table of Contents\nNVIDIA Corporation and Subs...
6,Step-Back,NVIDIA_Report_2024.pdf,38,0.8416,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,..."
7,Step-Back,NVIDIA_Report_2024.pdf,35,0.8704,Table of Contents\nFiscal Year 2024 Summary\n ...


who leads NVIDIA as its chief executive?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, O..."
1,Normal,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employ..."
2,Normal,NVIDIA_Report_2024.pdf,12,0.8722,"Table of Contents\nstrategy, planning, reporti..."
3,Normal,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management con..."
4,Step-Back,NVIDIA_Report_2024.pdf,11,0.5533,"Debora Shoquist 69 Executive Vice President, O..."
5,Step-Back,NVIDIA_Report_2024.pdf,11,0.6930,"supports diverse hiring, retention, and employ..."
6,Step-Back,NVIDIA_Report_2024.pdf,12,0.8751,"Table of Contents\nstrategy, planning, reporti..."
7,Step-Back,NVIDIA_Report_2024.pdf,12,0.9220,"Puri previously held marketing, management con..."


### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [26]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [27]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [53]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Step_back",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Step_back-9d98be61' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=157b1be7-969c-4e1e-8c16-20eca7822b54




0it [00:00, ?it/s]

what industries does NVIDIA operate in?
how can we compare the revenues of NVIDIA over different years?
who leads NVIDIA as its chief executive?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,35,0.5506,Revenue for fiscal year 2024 was $60.9 billion...
1,Normal,NVIDIA_Report_2024.pdf,35,0.6613,other NVIDIA RTX technologies.\nProfessional V...
2,Normal,NVIDIA_Report_2024.pdf,49,0.6642,Table of Contents\nNVIDIA Corporation and Subs...
3,Normal,NVIDIA_Report_2024.pdf,35,0.7459,Table of Contents\nFiscal Year 2024 Summary\n ...
4,Step-Back,NVIDIA_Report_2024.pdf,35,0.7436,Revenue for fiscal year 2024 was $60.9 billion...
5,Step-Back,NVIDIA_Report_2024.pdf,49,0.7951,Table of Contents\nNVIDIA Corporation and Subs...
6,Step-Back,NVIDIA_Report_2024.pdf,38,0.8416,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,..."
7,Step-Back,NVIDIA_Report_2024.pdf,35,0.8704,Table of Contents\nFiscal Year 2024 Summary\n ...


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,4,0.5249,infotainment systems; and Omniverse Enterprise...
1,Normal,NVIDIA_Report_2024.pdf,3,0.6685,serve. While the computing requirements of the...
2,Normal,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers..."
3,Normal,NVIDIA_Report_2024.pdf,6,0.7180,layers. This full-stack innovation approach al...
4,Step-Back,NVIDIA_Report_2024.pdf,4,0.6737,infotainment systems; and Omniverse Enterprise...
5,Step-Back,NVIDIA_Report_2024.pdf,4,0.6748,"Professional artists, architects and designers..."
6,Step-Back,NVIDIA_Report_2024.pdf,33,0.7028,NVIDIA pioneered accelerated computing to help...
7,Step-Back,NVIDIA_Report_2024.pdf,77,0.7338,The Graphics segment includes GeForce GPUs for...


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, O..."
1,Normal,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employ..."
2,Normal,NVIDIA_Report_2024.pdf,12,0.8723,"Table of Contents\nstrategy, planning, reporti..."
3,Normal,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management con..."
4,Step-Back,NVIDIA_Report_2024.pdf,11,0.5532,"Debora Shoquist 69 Executive Vice President, O..."
5,Step-Back,NVIDIA_Report_2024.pdf,11,0.6929,"supports diverse hiring, retention, and employ..."
6,Step-Back,NVIDIA_Report_2024.pdf,12,0.8749,"Table of Contents\nstrategy, planning, reporti..."
7,Step-Back,NVIDIA_Report_2024.pdf,12,0.9219,"Puri previously held marketing, management con..."


## Complete Evaluation

In [28]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Step_back",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Step_back-4d3bff35' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=1351ed53-dab8-4684-ba57-e84ac84cce2f




0it [00:00, ?it/s]

what factors can lead to significant changes in a company's total assets over a year?
What factors can influence a company's cash balance despite an increase in operating cash flow, and what might this indicate about its financial strategy and priorities?
What is the career background of NVIDIA's CFO?
What are the potential impacts of geopolitical conflicts on a company's operations and workforce?
how do companies ensure the safety and effectiveness of their autonomous vehicle technologies?
what are the different expenses that companies like NVIDIA typically incur, and how do these expenses relate to their revenue over time?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,51,0.6125,Table of Contents\nNVIDIA Corporation and Subs...
1,Normal,NVIDIA_Report_2024.pdf,53,0.7120,Table of Contents\nNVIDIA Corporation and Subs...
2,Normal,NVIDIA_Report_2024.pdf,49,0.7655,Table of Contents\nNVIDIA Corporation and Subs...
3,Normal,NVIDIA_Report_2024.pdf,50,0.7868,Table of Contents\nNVIDIA Corporation and Subs...
4,Step-Back,NVIDIA_Report_2024.pdf,23,1.1404,• changes in financial accounting standards or...
5,Step-Back,NVIDIA_Report_2024.pdf,68,1.1565,"Property, equipment and intangible assets acqu..."
6,Step-Back,NVIDIA_Report_2024.pdf,21,1.1823,"target company, technology may evolve and make..."
7,Step-Back,NVIDIA_Report_2024.pdf,54,1.1870,"taxes, goodwill, stock-based compensation, lit..."


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,34,0.7512,networking products has not experienced any si...
1,Normal,NVIDIA_Report_2024.pdf,21,0.7664,networking products has not experienced any si...
2,Normal,NVIDIA_Report_2024.pdf,34,0.7741,manage product availability and costs with our...
3,Normal,NVIDIA_Report_2024.pdf,21,0.7991,Table of Contents\nWe are monitoring the impac...
4,Step-Back,NVIDIA_Report_2024.pdf,20,0.8019,adverse political developments in or around an...
5,Step-Back,NVIDIA_Report_2024.pdf,34,0.8722,"Macroeconomic Factors\nMacroeconomic factors, ..."
6,Step-Back,NVIDIA_Report_2024.pdf,34,0.9178,networking products has not experienced any si...
7,Step-Back,NVIDIA_Report_2024.pdf,20,0.9187,"companies with Israeli operations, including b..."


...

what are the main areas where NVIDIA operates and offers its services?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,11,0.6106,"Debora Shoquist 69 Executive Vice President, O..."
1,Normal,NVIDIA_Report_2024.pdf,11,0.7513,"supports diverse hiring, retention, and employ..."
2,Normal,NVIDIA_Report_2024.pdf,12,0.8337,"Table of Contents\nstrategy, planning, reporti..."
3,Normal,NVIDIA_Report_2024.pdf,12,0.8522,"Puri previously held marketing, management con..."
4,Step-Back,NVIDIA_Report_2024.pdf,11,0.6252,"Debora Shoquist 69 Executive Vice President, O..."
5,Step-Back,NVIDIA_Report_2024.pdf,11,0.6887,"supports diverse hiring, retention, and employ..."
6,Step-Back,NVIDIA_Report_2024.pdf,12,0.8972,"Table of Contents\nstrategy, planning, reporti..."
7,Step-Back,NVIDIA_Report_2024.pdf,12,0.9158,"Puri previously held marketing, management con..."


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,4,0.5249,infotainment systems; and Omniverse Enterprise...
1,Normal,NVIDIA_Report_2024.pdf,3,0.6685,serve. While the computing requirements of the...
2,Normal,NVIDIA_Report_2024.pdf,4,0.6938,"Professional artists, architects and designers..."
3,Normal,NVIDIA_Report_2024.pdf,6,0.7180,layers. This full-stack innovation approach al...
4,Step-Back,NVIDIA_Report_2024.pdf,4,0.6152,"Professional artists, architects and designers..."
5,Step-Back,NVIDIA_Report_2024.pdf,33,0.6157,NVIDIA pioneered accelerated computing to help...
6,Step-Back,NVIDIA_Report_2024.pdf,4,0.6829,infotainment systems; and Omniverse Enterprise...
7,Step-Back,NVIDIA_Report_2024.pdf,77,0.7473,The Graphics segment includes GeForce GPUs for...


who leads NVIDIA as its chief executive?


,Query Type,Document Name,Page Number,Similarity Score,Retrieved Chunk
0,Normal,NVIDIA_Report_2024.pdf,11,0.5491,"Debora Shoquist 69 Executive Vice President, O..."
1,Normal,NVIDIA_Report_2024.pdf,11,0.6880,"supports diverse hiring, retention, and employ..."
2,Normal,NVIDIA_Report_2024.pdf,12,0.8722,"Table of Contents\nstrategy, planning, reporti..."
3,Normal,NVIDIA_Report_2024.pdf,12,0.9087,"Puri previously held marketing, management con..."
4,Step-Back,NVIDIA_Report_2024.pdf,11,0.5534,"Debora Shoquist 69 Executive Vice President, O..."
5,Step-Back,NVIDIA_Report_2024.pdf,11,0.6931,"supports diverse hiring, retention, and employ..."
6,Step-Back,NVIDIA_Report_2024.pdf,12,0.8751,"Table of Contents\nstrategy, planning, reporti..."
7,Step-Back,NVIDIA_Report_2024.pdf,12,0.9221,"Puri previously held marketing, management con..."
